In [8]:
import pandas as pd

In [9]:
# Load Datasets
etl_input = pd.read_parquet("./../datasets/etl_input.parquet", engine='fastparquet')
etl_output = pd.read_parquet("./../datasets/etl_output.parquet", engine='fastparquet')

In [10]:
# Row Count Validation
assert len(etl_input) == len(etl_output), "Row count mismatch!"

In [11]:
# Business Rule: Salary should not be negative
assert all(etl_output["salary"] >= 0), "Invalid salary values found!"

In [12]:
# Check for Missing Values
missing_values = etl_output.isnull().sum()
print("Missing Values: \n", missing_values)

Missing Values: 
 id                0
name              0
department        0
salary            0
ingestion_date    0
dtype: int64


In [13]:
# Data Transformation Check (e.g., added 'ingestion_date' column)
assert "ingestion_date" in etl_output.columns, "ETL Transformation missing!"

In [14]:
print("ETL Validation Passed!")

ETL Validation Passed!
